In [1]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd

sm_boto3 = boto3.client("sagemaker")  #we create a cliemty
sess = sagemaker.Session() #we are creating a seesion
region = sess.boto_region_name
bucket = "mypricebucket"   #anything i will store my data in
print("using bucket" + bucket)

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\test\AppData\Local\sagemaker\sagemaker\config.yaml
using bucketmypricebucket


# DATA INGESTION STARTS

### fEATURE eNGINEERING AND FEATURE PROCESS

In [2]:
df = pd.read_csv("vehicles_dataset.csv")

In [3]:
df.head()

,name,description,make,model,type,year,price,engine,cylinders,fuel,mileage,transmission,trim,body,doors,exterior_color,interior_color,drivetrain
0,2024 Jeep Wagoneer Series II,"\n \n Heated Leather Seats, Nav Sy...",Jeep,Wagoneer,New,2024,74600.0,24V GDI DOHC Twin Turbo,6.0,Gasoline,10.0,8-Speed Automatic,Series II,SUV,4.0,White,Global Black,Four-wheel Drive
1,2024 Jeep Grand Cherokee Laredo,Al West is committed to offering every custome...,Jeep,Grand Cherokee,New,2024,50170.0,OHV,6.0,Gasoline,1.0,8-Speed Automatic,Laredo,SUV,4.0,Metallic,Global Black,Four-wheel Drive
2,2024 GMC Yukon XL Denali,NaN,GMC,Yukon XL,New,2024,96410.0,"6.2L V-8 gasoline direct injection, variable v...",8.0,Gasoline,0.0,Automatic,Denali,SUV,4.0,Summit White,Teak/Light Shale,Four-wheel Drive
3,2023 Dodge Durango Pursuit,White Knuckle Clearcoat 2023 Dodge Durango Pur...,Dodge,Durango,New,2023,46835.0,16V MPFI OHV,8.0,Gasoline,32.0,8-Speed Automatic,Pursuit,SUV,4.0,White Knuckle Clearcoat,Black,All-wheel Drive
4,2024 RAM 3500 Laramie,\n \n 2024 Ram 3500 Laramie Billet...,RAM,3500,New,2024,81663.0,24V DDI OHV Turbo Diesel,6.0,Diesel,10.0,6-Speed Automatic,Laramie,Pickup Truck,4.0,Silver,Black,Four-wheel Drive


In [4]:
df.shape

(1002, 18)

In [5]:
df['price'].value_counts(normalize=True)

price
46835.0    0.013279
47340.0    0.010215
28860.0    0.006129
54949.0    0.005107
47785.0    0.004086
             ...   
34492.0    0.001021
36000.0    0.001021
53160.0    0.001021
59116.0    0.001021
48995.0    0.001021
Name: proportion, Length: 859, dtype: float64

In [6]:
df.columns

Index(['name', 'description', 'make', 'model', 'type', 'year', 'price',
       'engine', 'cylinders', 'fuel', 'mileage', 'transmission', 'trim',
       'body', 'doors', 'exterior_color', 'interior_color', 'drivetrain'],
      dtype='object')

#### to find the percentage of value that is missing

In [7]:
df.isnull().mean() * 100

name               0.000000
description        5.588822
make               0.000000
model              0.000000
type               0.000000
year               0.000000
price              2.295409
engine             0.199601
cylinders         10.479042
fuel               0.698603
mileage            3.393214
transmission       0.199601
trim               0.099800
body               0.299401
doors              0.698603
exterior_color     0.499002
interior_color     3.792415
drivetrain         0.000000
dtype: float64

In [8]:
# all my features in form of list
features = list(df.columns)
features

['name',
 'description',
 'make',
 'model',
 'type',
 'year',
 'price',
 'engine',
 'cylinders',
 'fuel',
 'mileage',
 'transmission',
 'trim',
 'body',
 'doors',
 'exterior_color',
 'interior_color',
 'drivetrain']

In [9]:
#remove just the price feature and make it your x
label = features.pop(-10)
label

'cylinders'

In [10]:
x = df[features]
y = df[label]

In [11]:
x.head()

,name,description,make,model,type,year,price,engine,fuel,mileage,transmission,trim,body,doors,exterior_color,interior_color,drivetrain
0,2024 Jeep Wagoneer Series II,"\n \n Heated Leather Seats, Nav Sy...",Jeep,Wagoneer,New,2024,74600.0,24V GDI DOHC Twin Turbo,Gasoline,10.0,8-Speed Automatic,Series II,SUV,4.0,White,Global Black,Four-wheel Drive
1,2024 Jeep Grand Cherokee Laredo,Al West is committed to offering every custome...,Jeep,Grand Cherokee,New,2024,50170.0,OHV,Gasoline,1.0,8-Speed Automatic,Laredo,SUV,4.0,Metallic,Global Black,Four-wheel Drive
2,2024 GMC Yukon XL Denali,NaN,GMC,Yukon XL,New,2024,96410.0,"6.2L V-8 gasoline direct injection, variable v...",Gasoline,0.0,Automatic,Denali,SUV,4.0,Summit White,Teak/Light Shale,Four-wheel Drive
3,2023 Dodge Durango Pursuit,White Knuckle Clearcoat 2023 Dodge Durango Pur...,Dodge,Durango,New,2023,46835.0,16V MPFI OHV,Gasoline,32.0,8-Speed Automatic,Pursuit,SUV,4.0,White Knuckle Clearcoat,Black,All-wheel Drive
4,2024 RAM 3500 Laramie,\n \n 2024 Ram 3500 Laramie Billet...,RAM,3500,New,2024,81663.0,24V DDI OHV Turbo Diesel,Diesel,10.0,6-Speed Automatic,Laramie,Pickup Truck,4.0,Silver,Black,Four-wheel Drive


In [12]:
y.head()

0    6.0
1    6.0
2    8.0
3    8.0
4    6.0
Name: cylinders, dtype: float64

In [13]:
#to find the number of records
x.shape

(1002, 17)

In [14]:
y.value_counts()

cylinders
4.0    502
6.0    277
8.0     89
3.0     27
0.0      2
Name: count, dtype: int64

In [15]:
#trainig
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.15, random_state=0)

In [16]:
#to see our shape
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


(851, 17)
(151, 17)
(851,)
(151,)


In [17]:
#put your traiining data unto a dataframe
trainX = pd.DataFrame(x_train)
trainX[label] = y_train

testX = pd.DataFrame(x_test)
trainX[label] = y_test

In [18]:
print(trainX.shape)
print(testX.shape)

(851, 18)
(151, 17)


In [19]:
trainX.head()

,name,description,make,model,type,year,price,engine,fuel,mileage,transmission,trim,body,doors,exterior_color,interior_color,drivetrain,cylinders
30,2024 Nissan Murano SV,\n \n This 2024 Nissan Murano SV w...,Nissan,Murano,New,2024,39867.0,24V MPFI DOHC,Gasoline,2.0,Automatic CVT,SV,SUV,4.0,White,Graphite,Front-wheel Drive,NaN
380,2023 Nissan Titan SV,\n \n Hurry and take advantage now...,Nissan,Titan,New,2023,46672.0,"ce 5.6L V-8 gasoline direct injection, DOHC, v...",Gasoline,1.0,Automatic,SV,Pickup Truck,4.0,Super Black,Black,Four-wheel Drive,NaN
758,2024 Chrysler Voyager LX,"3rd Row Seat,Keyless Start,Rear Spoiler,Black/...",Chrysler,Voyager,New,2024,38954.0,24V MPFI DOHC,Gasoline,8.0,9-Speed Automatic,LX,Passenger Van,4.0,Bright White Clearcoat,Black,Front-wheel Drive,NaN
856,2024 Jeep Grand Cherokee Limited,\n \n 2024 Jeep Grand Cherokee Lim...,Jeep,Grand Cherokee,New,2024,44759.0,24V MPFI DOHC,Gasoline,6.0,8-Speed Automatic,Limited,SUV,4.0,Bright White Clearcoat,Global Black,Four-wheel Drive,NaN
724,2024 Hyundai Tucson Hybrid Limited,Black Pearl 2024 Hyundai Tucson Hybrid Limited...,Hyundai,Tucson Hybrid,New,2024,40754.0,"ream 1.6L I-4 gasoline direct injection, DOHC,...",Hybrid,12.0,Automatic,Limited,SUV,4.0,Black Pearl,Black,All-wheel Drive,NaN


In [20]:
testX.isnull().sum()

name               0
description       10
make               0
model              0
type               0
year               0
price              4
engine             0
fuel               0
mileage            5
transmission       1
trim               0
body               0
doors              0
exterior_color     0
interior_color     6
drivetrain         0
dtype: int64

### Now we splitt the files into train and test csv file so we can push them to s3 bucket

In [21]:
trainX.to_csv("train.1.csv", index=False)
testX.to_csv("test.1.csv", index=False)



##### we send data to s3 bucket

In [22]:
#senddata to s3.  Sagemaker will take training data from s3
sk_prefix ="sagemaker/vehicles_dataset/sklearncontainer"
trainpath = sess.upload_data(path="train.1.csv", bucket=bucket,key_prefix=sk_prefix)

testpath = sess.upload_data(path="test.1.csv", bucket=bucket, key_prefix=sk_prefix)
print(trainpath)
print(testpath)

s3://mypricebucket/sagemaker/vehicles_dataset/sklearncontainer/train.1.csv
s3://mypricebucket/sagemaker/vehicles_dataset/sklearncontainer/test.1.csv


## DATA INGESTION ENDS

In [23]:
%%writefile script.py  

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, prediction_score
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO
import argparse
import os
import numpy as np
import pandas as pd



def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))  #the function loads the model from the model_dir
    return clf
    
if __name__ == "__main__": #execution starts from here, amd these arguments are required by default
    print("[INFO] Extrating arguments")
    parser = argparse.ArgumentParser()
    
    #hyperparameters sent by the client are passed as command line arguments 
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)
    
    #Data, model, and output directories
    parser.add_argument("--model_dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--output_dir", type=str, default="test.1.csv")
    parser.add_argument("--output_dir", type=str, default="train.1.csv")
    
    args, _ = parser.parse_known_args()
    
    
    print("SkLearn version:".sklearn.__version__)
    print("SKLearn version:".joblib.__version__)
    
    print("[INFO] Reading data")
    print()
    
    #load the data from the s3 bucket
    train_df = pd.read_csv(os.path.join(args.train, args.train_file ))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))
    
    features  = list(train_df.columns)
    label = features.pop(-1)
    
    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]
    
    print("Column order:")
    print(features)
    print()
    
    print("label column is : ", label)
    print()
    
    print("Data Shape: ")
    print()
    
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)    
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
    print("Training RandomForest Model....")
    print()
    model = RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state,vars=args.vars)
    model.fit(X_train, y_train)
    print()
    
    model_path = os.path.join(args.model_dir, "model.job") 
    joblib.dump(model, model_path)
    print("Model persisted at + model_path")
    print()
    
    
    y_pred = model.predict(X_test)    
    test_acc = accuracy_score(y_test, y_pred)
    test_rep = classification_report(y_test, y_pred)
    
    print()
    print("----MERTICS RESULTS FOR TESTING DATA----")
    print()
    print("Total Rows Are: ", X_test.shape[0])
    print("[TESTING] Model Accuracy: ", test_acc)
    print("[TESTING] Testing Report: ")
    print(test_rep)    
    
    

Overwriting script.py


In [24]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role="arn:aws:iam::953205729572:role/danny-role",
    instance_count=1,
    instance_type="ml.m5.xlarge",
    framework_version=FRAMEWORK_VERSION,
    py_version="py3",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    }
    
)

use_spot_instances=True,
max_wait = 7200,
max_run = 3600



In [25]:
#launch training job, with asynchronous call
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)
#sklearn_estimator.fit({"train": datapath}, wait=True)

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2024-06-04-05-51-48-089


2024-06-04 05:51:49 Starting - Starting the training job

...
2024-06-04 05:52:04 Starting - Preparing the instances for training...
2024-06-04 05:52:49 Downloading - Downloading the training image...
2024-06-04 05:53:09 Training - Training image download completed. Training in progress..2024-06-04 05:53:17,533 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-06-04 05:53:17,536 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-04 05:53:17,576 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-06-04 05:53:17,744 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-04 05:53:17,756 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-04 05:53:17,767 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-04 05:53:17,776 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_paramet

UnexpectedStatusException: Error for Training job sagemaker-scikit-learn-2024-06-04-05-51-48-089: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_containers/_trainer.py", line 84, in train
    entrypoint()
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 39, in main
    train(environment.Environment())
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 35, in train
    runner_type=runner.ProcessRunnerType)
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/entry_point.py", line 100, in run
    wait, capture_error
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/process.py", line 291, in run
    cwd=environment.code_dir,
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/process.py", line 208, in check_error
    info=extra_info,
sagemaker_training.errors.E